### Load and preprocess the dataset

In [11]:
import pandas as pd

path = '/home/azureuser/cloudfiles/code/Users/francesca/Synthetic-Data-Evaluation/uk_anonym/workshop/players_22.csv'
df = pd.read_csv(path)

/tmp/ipykernel_10629/3005243549.py:4: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


In [12]:
cols_to_keep = ['short_name',
        'player_positions',
        'overall',
        'wage_eur',
        'age',
        'height_cm',
        'weight_kg',
        'club_name',
        'nationality_name']

df = df[cols_to_keep]

In [13]:
positions = {
    'GoalKeeper' : ['GK'], # GoalKeeper
    'Defense' : ['CB', # Center Back
                 'LB', # Left Back
                 'RB', # Right Back
                 'LWB', # Left Wing Back
                 'RWB', # Right Wing Back
                 'CDM'], # Center Defensive Midfielder
    'Midfield' : ['CM', # Center Midfielder
                  'CAM', # Center Attacking Midfielder
                  'LM', # Left Midfielder
                  'RM'], # Right Midfielder
    'Offense' : ['LW', # Left Wing
                 'RW', # Right Wing
                 'CF', # Center Forward
                 'ST'] # Striker
}

def determine_position(specific_positions):
    counts = {pos: 0 for pos in positions}
    for sp in specific_positions.split(', '):
        for p, p_list in positions.items():
            if sp in p_list:
                counts[p] += 1
    max_pos = max(counts, key=counts.get)
    
    return max_pos

df['player_positions'] = df['player_positions'].apply(determine_position)
df.rename(columns={'player_positions': 'position'}, inplace=True)

In [14]:
df.dropna(inplace=True)

In [15]:
df['nationality_name'].value_counts()

England             1719
Germany             1214
Spain               1086
France               980
Argentina            960
                    ... 
Ethiopia               1
Papua New Guinea       1
Kyrgyzstan             1
Guam                   1
Indonesia              1
Name: nationality_name, Length: 163, dtype: int64

In [16]:
# same_name = df[df.duplicated(subset=['short_name'], keep=False)]
# same_name.sort_values(by='short_name')

idx = df.groupby(['short_name'])['overall'].idxmax()
df = df.loc[idx]
same_name = df[df.duplicated(subset=['short_name'], keep=False)]
df

,short_name,position,overall,wage_eur,age,height_cm,weight_kg,club_name,nationality_name
891,A Lan,Offense,77,27000.0,31,178,64,Guangzhou FC,China PR
19017,A. Abaz,GoalKeeper,50,500.0,19,187,76,FC St. Gallen 1879,Switzerland
10478,A. Abdallah,Defense,65,2000.0,23,170,68,Grenoble Foot 38,Comoros
13859,A. Abdellaoui,Defense,62,5000.0,28,183,73,Ettifaq FC,Algeria
5400,A. Abdi,Defense,70,4000.0,27,180,73,Stade Malherbe Caen,Tunisia
...,...,...,...,...,...,...,...,...,...
16650,Ș. Fara,GoalKeeper,58,500.0,20,189,77,FC Dinamo 1948 Bucureşti,Romania
17351,Ș. Pănoiu,Midfield,57,550.0,18,177,71,Rapid București,Romania
869,Ș. Radu,Defense,77,48000.0,34,183,79,Lazio,Romania
14764,Ș. Târnovanu,GoalKeeper,61,1000.0,21,197,87,FCSB (Steaua),Romania


In [17]:
df = df.sample(frac=1).reset_index(drop=True)
df

,short_name,position,overall,wage_eur,age,height_cm,weight_kg,club_name,nationality_name
0,I. Lawlor,GoalKeeper,61,1000.0,26,193,80,Dundee FC,Republic of Ireland
1,G. Benavídez,Defense,65,3000.0,25,175,71,Arsenal de Sarandí,Argentina
2,Óscar,Midfield,75,21000.0,23,174,68,Sevilla FC,Spain
3,B. Bolla,Defense,64,10000.0,21,180,72,Grasshopper Club Zürich,Hungary
4,M. Kjaergaard,Midfield,62,2000.0,18,189,70,FC Red Bull Salzburg,Denmark
...,...,...,...,...,...,...,...,...,...
18082,F. Altamirano,GoalKeeper,66,3000.0,25,186,78,Club Atlético Banfield,Argentina
18083,J. Güemez,Defense,71,13000.0,29,176,80,Club Atlético de San Luis,Mexico
18084,D. Criscito,Defense,76,16000.0,34,183,75,Genoa,Italy
18085,L. Paolucci,Midfield,62,3000.0,24,174,66,Royale Union Saint-Gilloise,Italy


In [18]:
df.to_csv('/home/azureuser/cloudfiles/code/Users/francesca/Synthetic-Data-Evaluation/uk_anonym/workshop/fifa.csv', index=False)